# SEIR model example
\begin{align}
\dot{S} & = S_f - \alpha\beta\frac{SI}{N+k_I I+k_R R} + r_{R\_S} R\\
\dot{E} & = E_f + \alpha\beta\frac{SI}{N+k_I I+k_R R} - E\frac{1}{t_{E\_I}} \\
\dot{I} & = E\frac{1}{t_{E\_I}} - I\frac{1}{t_{I\_R}} \\
\dot{R} & = I\frac{1}{t_{I\_R}} - r_{I\_R} R\\
\end{align}
Where:  
* $S:$ Susceptible
* $E:$ Exposed
* $I:$ Infectious
* $R:$ Removed
* $\alpha:$ Mobilty
* $\beta:$ Infection rate
* $N:$ Total population
* $t_{E\_I}:$ # Transition time between exposed and infectious
* $t_{I\_R}:$ # Transition time between infectious and recovered
* $r_{R\_S}:$ Immunity loss rate ($\frac{1}{t_{R\_S}}$)  
* $S_f,E_f,I_f,R_f:$ External flux
* $k_I:$ Infected saturation   
* $k_R:$ Immunity shield    


In [1]:
# Util libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
# Adding lib paths
import sys
sys.path.insert(1, '../src2/models/')

# cv19 libraries
from SEIR import SEIR
import toml
import copy

In [2]:
# For pop-up plots execute this code (optional)
import platform
OS = platform.system()

if OS == 'Linux':    
    %matplotlib tk
    print('Linux')
elif OS == 'Windows':
    %matplotlib qt
    print('Windows')
elif OS == 'Darwin':
    %matplotlib tk
    print('Mac (Funciona?)')

Linux


In [39]:
# Input configuration file
config = 'cfg/SEIR.toml'

In [40]:
cfg = toml.load(config)

# Beta

## Beginning

In [41]:
var = [0.8,0.9,1,1.1,1.2]
simcfg = [copy.deepcopy(cfg) for i in var]

In [42]:
for i in range(len(simcfg)):
    simcfg[i]['parameters']['dynamic']['beta']=cfg['parameters']['dynamic']['beta']*var[i]

In [43]:
%%capture
model = SEIR(config=config)
model.integrate()
peak = max(model.I)

In [44]:
# Build simulation object
sim = []
for i in simcfg:
    sim.append(SEIR(config = i))

No external data added
SEIR object created
No external data added
SEIR object created
No external data added
SEIR object created
No external data added
SEIR object created
No external data added
SEIR object created


In [45]:
%%capture
# Simulate (solve ODE)
for i in sim:
    i.integrate()

In [46]:
for i in range(len(sim)):
    plt.plot(sim[i].t,sim[i].I/peak,label='Beta: '+str(int(var[i]*100))+'%')
plt.xlim(0,300)
plt.legend(loc=0)
plt.title('Beta')
plt.ylabel('Infectious')
plt.xlabel('day')
plt.show()

## During Simulation

In [47]:
var2 = [0.8,0.9,1,1.1,1.2]
simcfg2 = [copy.deepcopy(cfg) for i in var2]

In [48]:
alpha = []
for i in var2:
    alpha.append('{"function":"Events","values":[1,'+str(i)+'],"days":[[0,30],[30,500]]}')

In [49]:
for i in range(len(simcfg2)):
    simcfg2[i]['parameters']['dynamic']['alpha']=alpha[i]

In [50]:
%%capture
# Build simulation object
sim2 = []
for i in simcfg2:
    sim2.append(SEIR(config = i))

In [51]:
%%capture
# Simulate (solve ODE)
for i in sim2:
    i.integrate()

In [76]:
for i in range(len(sim2)):
    plt.plot(sim2[i].t,sim2[i].I/peak,label='Beta: '+str(int(var2[i]*100))+'%')
plt.xlim(0,300)
plt.legend(loc=0)
plt.title('Beta')
plt.ylabel('Infectious')
plt.xlabel('day')
plt.axvline(20,linestyle='dashed',color='grey')
plt.show()

# Population

## Beginning

In [53]:
population = cfg['initialconditions']['population']

In [54]:
var3 = [0.8,0.9,1,1.1,1.2]
simcfg3 = [copy.deepcopy(cfg) for i in var3]

In [55]:
for i in range(len(simcfg3)):
    simcfg3[i]['initialconditions']['population']=population*var3[i]

In [56]:
%%capture
# Build simulation object
sim3 = []
for i in simcfg3:
    sim3.append(SEIR(config = i))

In [57]:
%%capture
# Simulate (solve ODE)
for i in sim3:
    i.integrate()

In [58]:
sim3

In [59]:
for i in range(len(sim3)):
    plt.plot(sim3[i].t,sim3[i].I/peak,label='Population: '+str(int(var3[i]*100))+'%')
plt.xlim(0,300)
plt.legend(loc=0)
plt.title('Population')
plt.ylabel('Infectious')
plt.xlabel('day')
plt.show()

## During Simulation

In [66]:
var4 = [-0.03,-0.015,0,0.015,0.03]

In [67]:
S_f = []
for i in var4:
    S_f.append('{"function":"Events","values":['+str(population*i)+'],"days":[[20,25]]}')

In [68]:
simcfg4 = [copy.deepcopy(cfg) for i in var4]
for i in range(len(simcfg4)):
    simcfg4[i]['parameters']['dynamic']['S_f']=S_f[i]

In [69]:
# Build simulation object
sim4 = []
for i in simcfg4:
    sim4.append(SEIR(config = i))

Executing Events
No external data added
SEIR object created
Executing Events
No external data added
SEIR object created
Executing Events
No external data added
SEIR object created
Executing Events
No external data added
SEIR object created
Executing Events
No external data added
SEIR object created


In [70]:
%%capture
# Simulate (solve ODE)
for i in sim4:
    i.integrate()

In [75]:
for i in range(len(sim)):
    plt.plot(sim4[i].t,sim4[i].I/peak,label='Population: '+str(int(var4[i]*100*10))+'%',linestyle='dashed')
plt.xlim(0,300)
plt.legend(loc=0)
plt.title('Population')
plt.ylabel('Infectious')
plt.xlabel('day')
plt.axvline(20,linestyle='dashed',color='grey')
plt.show()